In [1]:

from numpy import asarray
import tkinter as tk 
from tkinter import *
import sys
import os
import datetime
import cv2
import mysql.connector
import numpy as np
import cv2
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from os import listdir
from gtts import gTTS 
import os 
####Librerias para enviar mensaje
import smtplib
from email.message import EmailMessage
import mysql.connector
import time




hora = datetime.datetime.now()

cnx = mysql.connector.connect(user='root', password='1234567890',
                              host="localhost",
                              database='conector1',port=3307)

mycursor = cnx.cursor()

basededatos = "C:\\Users\\user\\Desktop\\Prueba1\\"

color = (67,67,67)

face_cascade = cv2.CascadeClassifier('C:\\Users\\user\\Desktop\\face_recognitionOpenCv2-master\\Cascades\\haarcascade_frontalcatface.xml')

#Inicio Ventana de interfaz.
ventana = Tk()
ventana.geometry('300x220')


palabra = StringVar()


def procesar_imagen(ruta_imagen):
    img = load_img(ruta_imagen, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def Cargarmodelo():
    model = Sequential()

    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))

    from keras.models import model_from_json
    model.load_weights('C:\\Users\\user\\Downloads\\vgg_face_weights.h5') #Cargamos los pesos desde vgg_face
    #.layers para tener la lista aplanada de mis capas, layers output lo mismo 
    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

    return vgg_face_descriptor

model = Cargarmodelo()

alumnos = dict()

for file in listdir(basededatos):
    alumno, extension = file.split(".")
    alumnos[alumno] = model.predict(procesar_imagen('C:\\Users\\user\\Desktop\\Prueba1\\%s.jpg' % (alumno)))[0,:]
    #Realizamos predicciones con nuestra base de datos. Aqui entrega nuestra probabilidad de pertenecia a cada clase


#Red siamesa para encontrar la distancia.
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

l = Label(ventana,text="Ingrese la matricula.")
l.pack()
nombre = Entry(ventana, textvariable=palabra).pack()

def foto():
    banaux = 0
    matricula=palabra.get()
    if len(matricula)==10 and matricula.isdigit()==True:
        sql =  "SELECT Matricula FROM alumnos_bachillerato WHERE Matricula ='"+str(matricula)+"'"
        mycursor.execute(sql)
        myresult = mycursor.fetchall()
        for x in myresult:
            auxiliar = x
            if matricula == '{}'.format(auxiliar[0]):
                cam = cv2.VideoCapture(0)     
                #print(matricula)
                face_detector = cv2.CascadeClassifier('C:\\Users\\user\\Desktop\\OpenCV-Face-Recognition-master\\FaceDetection\\Cascades\\haarcascade_frontalface_default.xml')
                count = 0
                while(True):
                    ret, img = cam.read()
                    faces = face_detector.detectMultiScale(img, 1.3, 5)
                    for (x,y,w,h) in faces:
                        print()
                        x1 = x
                        y1 = y
                        x2 = x+w
                        y2 = y+h
                        cv2.rectangle(img, (x1,y1), (x2,y2), (255,255,255), 2)     
                        count += 1
                        #Guardar las imagenes en la carpeta
                        cv2.imwrite("C:\\Users\\user\\Desktop\\Prueba1\\" + palabra.get() + ".jpg", img[y1:y2,x1:x2])
                        cv2.imshow('imagen', img)
                    k = cv2.waitKey(200) & 0xff
                    if k == 27:
                        break
                    elif count >= 1: 
                         break
                cam.release()
                cv2.destroyAllWindows() 
                
                l = Label(ventana,text="")
                l.pack()

def entrada():
    cap = cv2.VideoCapture(0) #camara
    auxiliar=0

    while(True):
        ret, img = cap.read()
        faces = face_cascade.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            if w > 130: 
                detected_face = img[int(y):int(y+h), int(x):int(x+w)] 
                detected_face = cv2.resize(detected_face, (224, 224)) #224x224

                img_pixels = image.img_to_array(detected_face)
                img_pixels = np.expand_dims(img_pixels, axis = 0)

                img_pixels /= 127.5
                img_pixels -= 1
                captured_representation = model.predict(img_pixels)[0,:]

                ban = 0
                for i in alumnos:
                    alumno_nombre = i
                    representation = alumnos[i] #Le paso las caracteristicas de mis alumnos

                    similarity = findCosineSimilarity(representation, captured_representation)
                    if(similarity < 0.30):
                        cv2.putText(img, alumno_nombre, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)


                        ban = 1
                        auxiliar = ban
                        if(ban==1):
                            sql =  "SELECT CorreoTutor FROM alumnos_bachillerato WHERE Matricula ='"+str(alumno_nombre)+"'"
                            mycursor.execute(sql)
                            myresult = mycursor.fetchall()
                            for x in myresult:
                                hola = x
                                sql1 =  "SELECT id FROM alumnos_bachillerato WHERE Matricula ='"+str(alumno_nombre)+"'"
                                matricula=str(alumno_nombre)
                                print(matricula)
                                mycursor.execute(sql1)
                                myresult1 = mycursor.fetchall()
                                for y in myresult1:
                                    hola1 = y
                                    sql2 =  "SELECT estado FROM registro_bachillerato WHERE id = {}".format(hola1[0])
                                    mycursor.execute(sql2)
                                    myresult2 = mycursor.fetchall()
                                    print(myresult2)
                                    for z in myresult2:
                                        hola2 = z
                                        if("{}".format(hola2[0]) == "Inactivo"):
                                            sql = "UPDATE conector1.alumnos_bachillerato SET Entrada = %s WHERE Matricula = %s" 
                                            hora = datetime.datetime.now()
                                            val = (hora,str(alumno_nombre))  
                                            mycursor.execute(sql,val)
                                            cnx.commit()
                                            
                                        else:
                                            sql = "UPDATE conector1.alumnos_bachillerato SET Entrada = %s WHERE Matricula = %s" 
                                            hora = datetime.datetime.now()
                                            val = (hora,str(alumno_nombre))
                                            mycursor.execute(sql,val)
                                            cnx.commit()
                                            
                                            
                                        




                        break


                #Si es correcto extraere el nombre de la base datos y pondre el texto.
                #cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
                #cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)



                #if(ban == 0): 
                    #cv2.putText(img, 'desconocido', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)





        cv2.imshow('img',img)

        if cv2.waitKey(1) & 0xFF == ord('q'): #q para salir
            break
        elif auxiliar == 1:
            break





    cap.release()
    cv2.destroyAllWindows()


 
                
       

    
    
    
def salida():
    cap = cv2.VideoCapture(0) #camara
    auxiliar=0
    while(True):
        ret, img = cap.read()
        faces = face_cascade.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            if w > 130: 
                detected_face = img[int(y):int(y+h), int(x):int(x+w)] 
                detected_face = cv2.resize(detected_face, (224, 224)) #224x224

                img_pixels = image.img_to_array(detected_face)
                img_pixels = np.expand_dims(img_pixels, axis = 0)

                img_pixels /= 127.5
                img_pixels -= 1
                captured_representation = model.predict(img_pixels)[0,:]

                ban = 0
                for i in alumnos:
                    alumno_nombre = i
                    representation = alumnos[i] #Le paso las caracteristicas de mis alumnos

                    similarity = findCosineSimilarity(representation, captured_representation)
                    if(similarity < 0.30):
                        cv2.putText(img, alumno_nombre, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)


                        ban = 1
                        auxiliar = ban
                        if(ban==1):
                            sql =  "SELECT CorreoTutor FROM alumnos_bachillerato WHERE Matricula ='"+str(alumno_nombre)+"'"
                            mycursor.execute(sql)
                            myresult = mycursor.fetchall()
                            for x in myresult:
                                hola = x
                                sql = "UPDATE conector1.alumnos_bachillerato SET Salida = %s WHERE Matricula = %s" 
                                hora = datetime.datetime.now()
                                val = (hora,str(alumno_nombre))
                                mycursor.execute(sql,val)
                                cnx.commit()
                               
                               








                        break


                #Si es correcto extraere el nombre de la base datos y pondre el texto.
                #cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
                #cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)



                #if(ban == 0): 
                    #cv2.putText(img, 'desconocido', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)





        cv2.imshow('img',img)

        if cv2.waitKey(1) & 0xFF == ord('q'): #q para salir
            break
        elif auxiliar == 1:
            break





    cap.release()
    cv2.destroyAllWindows()

 


    

    
    
    
    
    
    
c = Button(ventana, text="Capturar foto" , width=25, height=3,fg="black", command=foto)
c.pack()
    

b = Button(ventana, text="Reconocimiento Facial Entrada" , width=25, height=3,fg="black", command=entrada)
b.pack()


d = Button(ventana, text="Reconocimiento facial Salida!" , width=25, height=3, command=salida)
d.pack()

ventana.mainloop()

Using TensorFlow backend.
C:\Users\USER\AppData\Local\conda\conda\envs\proyecto\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\USER\AppData\Local\conda\conda\envs\proyecto\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\USER\AppData\Local\conda\conda\envs\proyecto\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16